In [1]:
!pip install kagglehub
!pip install dagshub
!pip install mlflow

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import kagglehub
import os
import string

import dagshub
import mlflow

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [3]:
dagshub.init(repo_owner='ayoub.boudra1', repo_name='deep-learning-model', mlflow=True)


Accessing as ayoub.boudra1

Initialized MLflow to track repo "ayoub.boudra1/deep-learning-model"

Repository ayoub.boudra1/deep-learning-model initialized!

In [4]:
path = kagglehub.dataset_download("ash2703/handsignimages")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/ash2703/handsignimages/versions/2


In [5]:
ALPHABET_LIST = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']
PATH = "/root/.cache/kagglehub/datasets/ash2703/handsignimages/versions/2/"
PATH_TRAIN = os.path.join(PATH, "Train")
PATH_TEST = os.path.join(PATH, "Test")


In [6]:
image_arr = []
image_name_file = []
image_name_value = []
def load_data(path):
    for letter in ALPHABET_LIST:
        letter_path = os.path.join(path, letter)
        for image_name in os.listdir(letter_path):
            image_path = os.path.join(letter_path, image_name)
            image = plt.imread(image_path)
            image_arr.append(image)
            image_name_file.append(image)
            image_name_value.append(letter)




In [7]:
def generate_data(X_data, y_data ):
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=False,
        fill_mode='nearest'
    )
    augmented_images = []
    augmented_targets = []
    augmentation_factor = 5

    for img, target in zip(X_data, y_data):
        img = img.reshape((1,) + img.shape)
        count = 0
        for batch in datagen.flow(img, batch_size=1):
            augmented_images.append(batch[0])
            augmented_targets.append(target)
            count += 1
            if count >= augmentation_factor:
                break
    augmented_images = np.array(augmented_images)
    augmented_targets = np.array(augmented_targets)

    X_data = np.concatenate((X_data, augmented_images), axis=0)
    y_data = np.concatenate((y_data, augmented_targets), axis=0)

    return X_data, y_data



In [8]:
def prepare_data(image_arr, image_name_value):
    X_data = np.array(image_arr)
    y_data = np.array(image_name_value)
    if len(X_data.shape) == 3:
        X_data = np.expand_dims(X_data, axis=-1)


    X_data = X_data.astype('float32') / 255
    label_to_int = {label: idx for idx, label in enumerate(np.unique(y_data))}
    y_data = np.array([label_to_int[label] for label in y_data])
    num_classes = len(label_to_int)
    y_data = to_categorical(y_data, num_classes=num_classes)
    return X_data, y_data



In [14]:
EPOCHS = 10
BATCH_SIZE = 32
LEARNING_RATE = 0.001

def train_model(X_data, y_data):
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)
    with mlflow.start_run():
      # Log custom parameters
      mlflow.log_param("epochs", EPOCHS)
      mlflow.log_param("batch_size", BATCH_SIZE)
      mlflow.log_param("learning_rate", LEARNING_RATE)


      input_shape = X_train.shape[1:]
      # Compile the model
      model = model = Sequential([
      Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
      MaxPooling2D(pool_size=(2, 2)),
      Dropout(0.25),
      Conv2D(64, kernel_size=(3, 3), activation='relu'),
      MaxPooling2D(pool_size=(2, 2)),
      Dropout(0.25),
      Flatten(),
      Dense(128, activation='relu'),
      Dropout(0.5),
      Dense(24, activation='softmax')  # Output layer matches number of classes
      ])
      model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

      # Train the model
      history = model.fit(X_train, y_train,
                          epochs=EPOCHS,
                          batch_size=BATCH_SIZE,
                          )

      # Evaluate the model
      test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
      mlflow.log_metric("test_loss", test_loss)
      mlflow.log_metric("test_accuracy", test_accuracy)

      # Log the model
      mlflow.keras.log_model(model, "cnn_model")
      model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
      return model

In [10]:
def evaluate_model(model, X_test, y_test):
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f'Test accuracy: {test_accuracy:.4f}')

    plt.plot(model.history['accuracy'], label='Training Accuracy')
    plt.plot(model.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

    plt.plot(model.history['loss'], label='Training Loss')
    plt.plot(model.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

    #confusion matrix
    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    y_test = np.argmax(y_test, axis=1)
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()


In [11]:
def save_model(model):
    model.save('asl_model.h5')
    print("Model saved successfully.")

In [12]:
def main():
    load_data(PATH_TRAIN)
    X_data, y_data = prepare_data(image_arr, image_name_value)
    X_data, y_data = generate_data(X_data, y_data)
    model = train_model(X_data, y_data)
    evaluate_model(model, X_data, y_data)
    save_model(model)

In [15]:
main()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
8237/8237 ━━━━━━━━━━━━━━━━━━━━ 33s 3ms/step - accuracy: 0.5403 - loss: 1.4448
Epoch 2/10
8237/8237 ━━━━━━━━━━━━━━━━━━━━ 33s 3ms/step - accuracy: 0.8695 - loss: 0.3848
Epoch 3/10
8237/8237 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step - accuracy: 0.9065 - loss: 0.2743
Epoch 4/10
8237/8237 ━━━━━━━━━━━━━━━━━━━━ 41s 3ms/step - accuracy: 0.9246 - loss: 0.2232
Epoch 5/10
8237/8237 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step - accuracy: 0.9347 - loss: 0.1954
Epoch 6/10
8237/8237 ━━━━━━━━━━━━━━━━━━━━ 40s 3ms/step - accuracy: 0.9410 - loss: 0.1762
Epoch 7/10
8237/8237 ━━━━━━━━━━━━━━━━━━━━ 42s 3ms/step - accuracy: 0.9458 - loss: 0.1643
Epoch 8/10
8237/8237 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.9493 - loss: 0.1539
Epoch 9/10
8237/8237 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step - accuracy: 0.9511 - loss: 0.1486
Epoch 10/10
8237/8237 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step - accuracy: 0.9534 - loss: 0.1418


2024/12/07 22:27:06 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/12/07 22:27:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Epoch 1/10
8237/8237 ━━━━━━━━━━━━━━━━━━━━ 28s 3ms/step - accuracy: 0.9553 - loss: 0.1381 - val_accuracy: 0.9968 - val_loss: 0.0121
Epoch 2/10
8237/8237 ━━━━━━━━━━━━━━━━━━━━ 27s 3ms/step - accuracy: 0.9554 - loss: 0.1365 - val_accuracy: 0.9971 - val_loss: 0.0121
Epoch 3/10
8237/8237 ━━━━━━━━━━━━━━━━━━━━ 41s 3ms/step - accuracy: 0.9568 - loss: 0.1328 - val_accuracy: 0.9976 - val_loss: 0.0094
Epoch 4/10
8237/8237 ━━━━━━━━━━━━━━━━━━━━ 40s 3ms/step - accuracy: 0.9577 - loss: 0.1319 - val_accuracy: 0.9972 - val_loss: 0.0102
Epoch 5/10
8237/8237 ━━━━━━━━━━━━━━━━━━━━ 42s 3ms/step - accuracy: 0.9586 - loss: 0.1290 - val_accuracy: 0.9971 - val_loss: 0.0110
Epoch 6/10
8237/8237 ━━━━━━━━━━━━━━━━━━━━ 28s 3ms/step - accuracy: 0.9591 - loss: 0.1284 - val_accuracy: 0.9981 - val_loss: 0.0078
Epoch 7/10
8237/8237 ━━━━━━━━━━━━━━━━━━━━ 42s 3ms/step - accuracy: 0.9606 - loss: 0.1238 - val_accuracy: 0.9983 - val_loss: 0.0083
Epoch 8/10
8237/8237 ━━━━━━━━━━━━━━━━━━━━ 39s 3ms/step - accuracy: 0.9601 - loss: 0

TypeError: 'History' object is not subscriptable